In [1]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load model + LoRA.
base_model = "Qwen/Qwen3-1.7B-Base"
adapter_path = "/Users/tanisha/Documents/cs-4130/project/qwen_sft_vr_run2/final"

tokenizer = AutoTokenizer.from_pretrained(adapter_path)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(model, adapter_path)
model.eval()

def generate_scene(prompt):
    full_prompt = prompt.strip() + "\nHTML_START\n"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=800,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

ui = gr.Interface(
    fn=generate_scene,
    inputs="text",
    outputs="text",
    title="VR Scene Generator",
    description="Enter a prompt and the fine-tuned Qwen model produces the HTML scene."
)

ui.launch()

ModuleNotFoundError: No module named 'gradio'